In [ ]:
#Métrica de saliência baseada em distribuição
#A divergência KL é não linear.
#Varia no intervalo de zero ao infinito.
#Um valor zero indica que as duas funções de densidade de probabilidade são estritamente iguais. 

import numpy as np
import matplotlib.pyplot as plt
import glob
import os

count = 0
q_all = []
p_all = []
out_all = []
media = 0
out = 0
EPSILON = np.finfo('float').eps

def normalize(x, method='standard', axis=None):
	x = np.array(x, copy=False)
	if axis is not None:
		y = np.rollaxis(x, axis).reshape([x.shape[axis], -1])
		shape = np.ones(len(x.shape))
		shape[axis] = x.shape[axis]
		if method == 'standard':
			res = (x - np.mean(y, axis=1).reshape(shape)) / np.std(y, axis=1).reshape(shape)
		elif method == 'range':
			res = (x - np.min(y, axis=1).reshape(shape)) / (np.max(y, axis=1) - np.min(y, axis=1)).reshape(shape)
		elif method == 'sum':
			res = x / np.float_(np.sum(y, axis=1).reshape(shape))
		else:
			raise ValueError('method not in {"standard", "range", "sum"}')
	else:
		if method == 'standard':
			res = (x - np.mean(x)) / np.std(x)
		elif method == 'range':
			res = (x - np.min(x)) / (np.max(x) - np.min(x))
		elif method == 'sum':
			res = x / float(np.sum(x))
		else:
			raise ValueError('method not in {"standard", "range", "sum"}')
	return res

def KLD(p, q):
	p = normalize(p, method='sum')
	q = normalize(q, method='sum')
	return np.sum(np.where(p != 0, p * np.log((p+EPSILON) / (q+EPSILON)), 0))


if __name__ == '__main__':
    folder = r'...\saliency_all' #Diretório que contenha os mapas de saliência e os grounds truth

    gt_saliency_folder = folder + '\salmap' #mapas de saliencia do modelo
    gt_fixation_folder = folder + '\\fixmap' #mapas de saliencia - ground truth
    #print(gt_saliency_folder)
     
            
    for filename_sal in glob.glob(os.path.join(gt_saliency_folder,'*.png')):
        #print(filename_sal)
        p= plt.imread(filename_sal)
        p_all.append(p)
    for filename_fix in glob.glob(os.path.join(gt_fixation_folder,'*.jpeg')):
        #print(filename_fix)
        q= plt.imread(filename_fix)
        q_all.append(p)              
            
    for i in range(0,len(p_all)):
        out = KLD(p_all[i],p_all[i])
        out_all.append(out)
    soma = sum(out_all)
    media = soma/len(p_all)
    print (media) 
             